# App Overview

In [ ]:
# 🛠 Imports and logging setup
import logging
import datetime
import pandas as pd
import yfinance as yf
import requests
from bs4 import BeautifulSoup
import datetime
import json
from yfinance import Ticker
from typing import List
from pandas import DataFrame
from datetime import datetime, timezone
from typing import List
from yfinance import Ticker
from tqdm import tqdm
from tqdm.auto import tqdm as with_progress

In [10]:
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

def log(msg, emoji="ℹ️"):
    logging.info(f"{emoji} {msg}")

In [ ]:
# 🌍 OBSOLETE: Define sectors & scrape tickers (placeholder: extend over time)
todays_date = datetime.date.today().strftime("%Y-%m-%d")
log(f"Fetching stock lists for {todays_date}", "📅")

# For demo: hardcode sample tickers (can extend by scraping)
tech_tickers = ["AAPL", "MSFT", "NVDA", "GOOGL", "AMZN"]
renewable_tickers = ["NEE", "ENPH", "PLUG", "FSLR", "SEDG"]
manufacturing_tickers = ["GE", "CAT", "DE", "BA", "HON"]

pruned_tickers = tech_tickers + renewable_tickers + manufacturing_tickers
log(f"Collected {len(pruned_tickers)} tickers (sample universe)", "📊")

universe_df = pd.DataFrame({
    "Ticker": pruned_tickers,
    "Sector": ["Tech"]*len(tech_tickers) + ["Renewable"]*len(renewable_tickers) + ["Manufacturing"]*len(manufacturing_tickers)
})
universe_df


📊 Investor Metric Table (with Examples)
| Metric                      | Short Name        | What it shows                               | Why you care (as an investor)                                   | Example (simple scenario)                                              | Typical Range | Red Flag 🚩        | Green Flag ✅  |
| --------------------------- | ----------------- | ------------------------------------------- | --------------------------------------------------------------- | ---------------------------------------------------------------------- | ------------- | ------------------ | ------------- |
| **Dividend Yield**          | Yield %           | Yearly cash payout vs share price           | It’s your “paycheck” for owning the stock.                      | Invest $100k → 5% yield pays you $5k yearly.                           | 2–6%          | >8% often unsafe   | 3–5% steady   |
| **P/E (Price-to-Earnings)** | Price ÷ Profit    | Price of share vs profit per share          | Tells if stock is cheap or expensive compared to what it earns. | P/E 15 = you pay $15 for each $1 of profit.                            | 10–20         | >30 without growth | 12–18         |
| **PEG**                     | Price vs Growth   | P/E compared to growth rate                 | Shows if a high price is justified by fast growth.              | Two stocks at P/E 30: one grows 30% (fair), one grows 5% (overpriced). | ~1            | >2 overpaying      | ~1 fair       |
| **Debt-to-Equity**          | Debt Load         | Debt vs company’s own money                 | High debt makes a company fragile; lenders get paid before you. | $2 debt for each $1 own money = risky.                                 | 0.5–2         | >3 fragile         | <1 safe       |
| **Net Debt / EBITDA**       | Debt Years        | Debt minus cash vs yearly earnings          | How many years of profit needed to clear debt.                  | Debt $10B, profit $2B = 5 years to pay off.                            | 2–4           | >5 heavy risk      | <2 strong     |
| **EV/EBITDA**               | Payback Years     | Company’s full price tag vs yearly earnings | How long it takes to “earn back” the company price.             | Company valued $100B, earns $10B/yr → 10 years.                        | 6–10          | >15 overpriced     | 6–8 fair      |
| **EV/Sales**                | Price vs Sales    | Company’s full price tag vs yearly sales    | Lets you value sales even if no profits.                        | Valued $5B, sales $2B → 2.5× sales.                                    | 1–3           | >5 hype            | <2 reasonable |
| **ROE (Return on Equity)**  | Profit Efficiency | Profit vs company’s own money               | Shows how well company turns its resources into profit.         | Equity $10B, profit $2B = 20%.                                         | 10–20%        | <5% weak           | >15% strong   |
| **Current Ratio**           | Liquidity         | Assets vs short-term bills                  | Shows if company can cover near-term bills.                     | $2 assets for each $1 bill due = safe.                                 | 1.0–2.0       | <1 cash crunch     | 1.5–2 cushion |


In [257]:
# simple in-memory cache { "YYYY-MM-DD": { "TICKER": TickerObj } }
stock_mem_cache: dict[str, dict[str, yf.Ticker]] = {}

In [ ]:
def get_stock(ticker_symbol:str)->Ticker:
    global stock_mem_cache

    todays_date = datetime.today().date().isoformat()  # YYYY-MM-DD
    if todays_date not in stock_mem_cache:
        stock_mem_cache[todays_date] = {}
    if ticker_symbol not in stock_mem_cache[todays_date]:
        #log(f"Stock cache miss for '{ticker_symbol}' with date '{todays_date}'", "⚠️")
        stock_mem_cache[todays_date][ticker_symbol] = yf.Ticker(ticker_symbol)
    return stock_mem_cache[todays_date][ticker_symbol]


In [259]:
# Clear cache
def clear_cache():
    global stock_mem_cache
    stock_mem_cache = {}    

In [260]:
clear_cache()

In [262]:
# 🌍 Fetch top 50 live high dividend tickers (via screener + yield filter)
today_str = datetime.today().strftime("%Y-%m-%d")
log(f"Fetching live high dividend stocks for {today_str}", "📅")

import requests, yfinance as yf

gross_ticker_count = 200
scrId = "most_actives"  # wide enough universe
api_url = (
    f"https://query2.finance.yahoo.com/v1/finance/screener/predefined/saved?"
    f"formatted=true&scrIds={scrId}&count={gross_ticker_count}&start=0"
)

resp = requests.get(api_url, headers={"User-Agent": "Mozilla/5.0"})
data = resp.json()
quotes = (
    data.get("finance", {})
        .get("result", [{}])[0]
        .get("quotes", [])
)

log(f"Collected {len(quotes)} tickers from Yahoo screener '{scrId}'", "📊")

# ✅ Filter tickers by dividend yield
high_dividend_tickers = []
yield_floor = 3.0
low_yield_stock_cnt = 0
for quote in with_progress(quotes, desc=f"Searching for high dividend stocks above {yield_floor}%"):
    ticker_symbol = quote.get("symbol")
    if not ticker_symbol:
        continue
    try:
        stock = get_stock(ticker_symbol)
        stock_info = stock.info
        dy = stock_info.get("dividendYield")
        if dy and dy > yield_floor:  # only >3% dividend yield
            high_dividend_tickers.append(ticker_symbol)
            #log(f"{t} passes dividend filter ({dy:.2f}%)", "✅")
        else:
            low_yield_stock_cnt+=1
    except Exception as e:
        log(f"Problem with ticker '{ticker_symbol}'. Exception: {e}", "⚠️")

log(f"Found {len(high_dividend_tickers)} stocks above {yield_floor}%.", "🎯")
log(f"Skipped {low_yield_stock_cnt} stocks as they were below the yield floor of {yield_floor}%.", "⏭️")
pruned_tickers = high_dividend_tickers[:50]  # 🔑 <-- your original flow preserved
log(f"Final selection: {len(pruned_tickers)} high dividend tickers", "📉")

universe_df = pd.DataFrame({
    "Ticker": pruned_tickers,
    "Sector": "High Dividend"
})


2025-09-26 11:49:07,667 - INFO - 📅 Fetching live high dividend stocks for 2025-09-26
2025-09-26 11:49:08,441 - INFO - 📊 Collected 200 tickers from Yahoo screener 'most_actives'
Searching for high dividend stocks above 3.0%: 100%|██████████| 200/200 [01:25<00:00,  2.33it/s]
2025-09-26 11:50:34,415 - INFO - 🎯 Found 46 stocks above 3.0%.
2025-09-26 11:50:34,416 - INFO - ⏭️ Skipped 154 stocks as they were below the yield floor of 3.0%.
2025-09-26 11:50:34,417 - INFO - 📉 Final selection: 46 high dividend tickers


In [261]:
# TODO Debug
stock = get_stock("NVDA")

In [ ]:
universe_df

In [ ]:
# TODO: Seems to be obsolete
# 🌍 Fetch tickers from Yahoo screener (generic)
todays_date = datetime.date.today().strftime("%Y-%m-%d")
log(f"Fetching screener results for {todays_date}", "📅")

# You can change scrId here: "day_gainers", "most_actives", "day_losers", etc.
scrId = "day_gainers"

api_url = (
    f"https://query2.finance.yahoo.com/v1/finance/screener/predefined/saved?"
    f"formatted=true&scrIds={scrId}&count=50&start=0"
)

resp = requests.get(api_url, headers={"User-Agent": "Mozilla/5.0"})
log(f"HTTP status code: {resp.status_code}", "🌐")

data = resp.json()
quotes = (
    data.get("finance", {})
        .get("result", [{}])[0]
        .get("quotes", [])
)

log(f"Found {len(quotes)} quotes under screener '{scrId}'", "📑")

high_dividend_tickers = [
    {"Ticker": q.get("symbol"), "Name": q.get("longName", q.get("shortName"))}
    for q in quotes if q.get("symbol")
]

universe_df = pd.DataFrame(high_dividend_tickers)
universe_df["Sector"] = scrId
log(f"Collected {len(universe_df)} tickers from Yahoo screener '{scrId}'", "📊")

universe_df


In [245]:
data


[]

In [158]:
def calc_current_div_yield(stock: Ticker)->float:
    stock_info = stock.info or {}
    return round((stock_info.get("dividendRate")/stock_info.get("currentPrice"))*100,2)

In [226]:
def to_pct(raw):
    if raw is None:
        pct = None
    else:
        pct = round(raw * 100, 2) if raw <= 1 else round(raw, 2)
    return pct

In [232]:
def from_unix_datetime(ts: int):
    """Convert UNIX timestamp (seconds) → UTC timezone-aware datetime."""
    if ts is None:
        return None
    try:
        return datetime.fromtimestamp(int(ts), tz=timezone.utc)
    except Exception:
        return None

In [231]:
def to_date(unix_ts: int):
    """Convert UNIX timestamp (seconds) → UTC timezone-aware datetime."""
    if unix_ts is None:
        return None
    try:
        return datetime.fromtimestamp(int(unix_ts), tz=timezone.utc).date()
    except Exception:
        return None 

In [223]:
def print_stock_info(stock:Ticker):
    stock_info_json = stock.info or {}
    print(pretty_print_json(stock_info_json))
    print("=======================")
    print(f"\n🔍 {ticker_symbol} — {stock_info_json.get('longName')}")
    #print("dividendYield raw:", stock_info.get("dividendYield"))
    print("displayName:", stock_info_json.get("displayName"))
    print("symbol:", stock_info_json.get("symbol"))
    print("website:", stock_info_json.get("website"))
    print("exchange:", stock_info_json.get("exchange"))
    print("country:", stock_info_json.get("country"))
    print("fiveYearAvgDividendYield:", stock_info_json.get("fiveYearAvgDividendYield"))
    print("dividendRate:", stock_info_json.get("dividendRate")) # Expected
    print("dividendYield:", stock_info_json.get("dividendYield")) # Based on price between previous
    print(f"currentDividendYield:{calc_current_div_yield(stock):.2f}") # Forward Dividend Rate ÷ Current Share Price × 100 %.
    print("lastDividendDate:", to_date(stock_info_json.get("lastDividendDate")))
    print("dividendDate:", to_date(stock_info_json.get("dividendDate")))
    print("exDividendDate:", to_date(stock_info_json.get("exDividendDate")))
    print("lastDividendValue:", stock_info_json.get("lastDividendValue"))
    print("currentPrice:", stock_info_json.get("currentPrice"))
    print("quoteType:", stock_info_json.get("quoteType"))
    print("industry:", stock_info_json.get("industry"))
    #print("sharesOutstanding:", stock_info.get("sharesOutstanding"))
    #print("currency:", stock_info.get("currency"))
    #print("ask:", stock_info.get("ask"))
    #print("askSize:", stock_info.get("askSize"))
    #print("previousClose:", stock_info.get("previousClose"))
    #print("market:", stock_info.get("market"))
    #print("marketCap:", stock_info.get("marketCap"))
    #print("fiftyDayAverage:", stock_info.get("fiftyDayAverage"))
    #print("fiftyTwoWeekHigh:", stock_info.get("fiftyTwoWeekHigh"))
    #print("fiftyTwoWeekLow:", stock_info.get("fiftyTwoWeekLow"))
    #print("boardRisk:", stock_info.get("boardRisk"))

In [ ]:
def to_millions(amount)->float:
    modified_amount = None
    if(amount is not None and amount):
        if(amount > 1000000): modified_amount = round(amount / 1000000, ndigits=2)
        else: modified_amount = round(amount, ndigits=2)
    log(f"called to millions with amount={amount} and type '{type(amount)}'. Returning amount={modified_amount}")
    return round(modified_amount, ndigits=2)

In [280]:
def stocks_to_pd(stocks: List[Ticker])->DataFrame:
    rows = []
    #for stock in stocks:
    for stock in tqdm(stocks, desc="Converting stock list to DataFrame"):
        #stock = yf.Ticker(t)
        stock_info_json = stock.info or {}
        rows.append({
            "🔍": stock_info_json.get("longName"),
            "displayName": stock_info_json.get("displayName"),
            "symbol": stock_info_json.get("symbol"),
            "website": stock_info_json.get("website"),
            "exchange": stock_info_json.get("exchange"),
            "country": stock_info_json.get("country"),
            "fiveYearAvgDividendYield": stock_info_json.get("fiveYearAvgDividendYield"),
            "dividendRate": stock_info_json.get("dividendRate"),
            "dividendYield": stock_info_json.get("dividendYield"),
            "currentDividendYield": calc_current_div_yield(stock),
            "lastDividendDate": to_date(stock_info_json.get("lastDividendDate")),
            "dividendDate": to_date(stock_info_json.get("dividendDate")),
            "exDividendDate": to_date(stock_info_json.get("exDividendDate")),
            "lastDividendValue": stock_info_json.get("lastDividendValue"),
            "currentPrice": stock_info_json.get("currentPrice"),
            "quoteType": stock_info_json.get("quoteType"),
            "industry": stock_info_json.get("industry"),
            "sharesOutstanding": stock_info_json.get("sharesOutstanding"),
            "currency": stock_info_json.get("currency"),
            "ask": stock_info_json.get("ask"),
            "askSize": stock_info_json.get("askSize"),
            "previousClose": stock_info_json.get("previousClose"),
            "market": stock_info_json.get("market"),
            "marketCap": stock_info_json.get("marketCap"),
            "fiftyDayAverage": stock_info_json.get("fiftyDayAverage"),
            "fiftyTwoWeekHigh": stock_info_json.get("fiftyTwoWeekHigh"),
            "fiftyTwoWeekLow": stock_info_json.get("fiftyTwoWeekLow"),
            "boardRisk": stock_info_json.get("boardRisk"),
            "returnOnEquity": stock_info_json.get("returnOnEquity"),
            "grossProfits": to_millions(stock_info_json.get("grossProfis")),
            "totalEarnings": to_millions(stock_info_json.get("totalEarnings")),
            "totalDebt": to_millions(stock_info_json.get("totalDebt")),
            "totalCash": to_millions(stock_info_json.get("totalCash")),
            "ebitda": to_pct(stock_info_json.get("ebitda")),
        })

    return pd.DataFrame(rows)

In [283]:
# 💰 Download dividend + yield + totals (non-breaking, robust)
log("Downloading dividend, yields, and sharesOutstanding…", "⬇️")

data = []
stocks: List[Ticker] = []
try:
    for ticker_symbol in with_progress(pruned_tickers, desc="Fetching stock details"):
        stock = get_stock(ticker_symbol)
        stocks.append(stock)
        #log(f"Downloaded {ticker_symbol}", "✅")
        #print_stock_info(stock)
except Exception as e:
    log(f"Failed to download stock information for ticker symbol '{ticker_symbol}': {e}", "❌")

log(f"Converting stock list to dataframe...")
stocks_pd = stocks_to_pd(stocks)
log(f"Sorting dataframe...")
stocks_pd.sort_values("currentDividendYield", ascending=False, inplace=True)

    


2025-09-26 12:03:07,228 - INFO - ⬇️ Downloading dividend, yields, and sharesOutstanding…
Fetching stock details: 100%|██████████| 46/46 [00:00<00:00, 86519.28it/s]
2025-09-26 12:03:07,232 - INFO - ℹ️ Converting stock list to dataframe...
Converting stock list to DataFrame:   0%|          | 0/46 [00:00<?, ?it/s]2025-09-26 12:03:07,235 - INFO - ℹ️ called to millions with number 'None' and type '<class 'NoneType'>'
2025-09-26 12:03:07,235 - INFO - ℹ️ called to millions with number 'None' and type '<class 'NoneType'>'
2025-09-26 12:03:07,236 - INFO - ℹ️ called to millions with number '160238993408' and type '<class 'int'>'
2025-09-26 12:03:07,237 - INFO - ℹ️ called to millions with number '28280999936' and type '<class 'int'>'
2025-09-26 12:03:07,238 - INFO - ℹ️ called to millions with number 'None' and type '<class 'NoneType'>'
2025-09-26 12:03:07,239 - INFO - ℹ️ called to millions with number 'None' and type '<class 'NoneType'>'
2025-09-26 12:03:07,240 - INFO - ℹ️ called to millions with

In [272]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
stocks_pd

,🔍,displayName,symbol,website,exchange,country,fiveYearAvgDividendYield,dividendRate,dividendYield,currentDividendYield,lastDividendDate,dividendDate,exDividendDate,lastDividendValue,currentPrice,quoteType,industry,sharesOutstanding,currency,ask,askSize,previousClose,market,marketCap,fiftyDayAverage,fiftyTwoWeekHigh,fiftyTwoWeekLow,boardRisk,returnOnEquity,grossProfits,totalEarnings,totalDebt,totalCash,ebitda
16,AGNC Investment Corp.,AGNC Inv,AGNC,https://agnc.com,NMS,United States,13.11,1.44,14.75,14.75,2025-08-29,2025-10-09,2025-09-30,0.120,9.76,EQUITY,REIT - Mortgage,1053322495,USD,10.23,1,9.76,us_market,10280427520,9.7570,10.630,7.850,4.0,0.03979,None,None,69319.000064,22172.999680,NaN
11,Petróleo Brasileiro S.A. - Petrobras,None,PBR,https://petrobras.com.br,NYQ,Brazil,21.69,1.83,13.96,13.96,2025-08-25,2025-12-30,2025-08-25,0.247,13.11,EQUITY,Oil & Gas Integrated,3721115691,USD,0.00,235,13.45,us_market,84078272512,12.4978,15.340,11.030,NaN,0.18483,None,None,68064.002048,9500.999680,3.319075e+10
33,The Western Union Company,None,WU,https://www.westernunion.com,NYQ,United States,6.77,0.94,11.93,11.93,2025-09-16,2025-09-30,2025-09-16,0.235,7.88,EQUITY,Credit Services,322967281,USD,7.96,37,8.16,us_market,2544982272,8.3564,12.400,7.850,1.0,1.35322,None,None,2749.199872,1019.600000,9.037000e+08
12,Vale S.A.,Vale,VALE,https://www.vale.com,NYQ,Brazil,9.08,1.18,10.85,10.84,2025-08-13,2025-09-10,2025-08-13,0.342,10.89,EQUITY,Other Industrial Metals & Mining,4268778775,USD,0.00,536,10.83,us_market,46762217472,10.2286,12.050,8.060,NaN,0.12837,None,None,110776.999936,31086.999552,7.161200e+10
20,Stellantis N.V.,Stellantis,STLA,https://www.stellantis.com,NYQ,Netherlands,NaN,0.77,8.27,8.27,2025-04-23,2025-05-05,2025-04-23,0.770,9.31,EQUITY,Auto Manufacturers,2888724012,USD,0.00,310,9.63,us_market,27945707520,9.4798,16.290,8.390,4.0,-0.03058,None,None,40848.998400,30971.000832,2.251000e+09
40,"United Parcel Service, Inc.",United Parcel Service,UPS,https://www.ups.com,NYQ,United States,3.71,6.56,7.94,7.94,2025-08-18,2025-09-04,2025-08-18,1.640,82.58,EQUITY,Integrated Freight & Logistics,736042904,USD,82.74,3,83.90,us_market,69992300544,88.4406,145.010,82.000,7.0,0.34907,None,None,28909.000704,6286.000128,1.149700e+10
4,Ambev S.A.,Ambev,ABEV,https://www.ambev.com.br,NYQ,Brazil,4.40,0.18,7.64,7.83,2025-08-11,2025-10-14,2025-08-11,0.023,2.30,EQUITY,Beverages - Brewers,15592286429,USD,2.29,2156,2.29,us_market,36527362048,2.2642,2.640,1.760,NaN,0.16049,None,None,3157.764096,17524.578304,2.730567e+10
34,"Conagra Brands, Inc.",Conagra Brands,CAG,https://www.conagrabrands.com,NYQ,United States,4.11,1.40,7.74,7.74,2025-07-30,2025-08-28,2025-07-30,0.350,18.08,EQUITY,Packaged Foods,478693731,USD,0.00,23,18.53,us_market,8654782464,19.0298,32.900,18.065,1.0,0.13214,None,None,8310.600192,68.000000,2.070200e+09
36,Energy Transfer LP,None,ET,https://energytransfer.com,NYQ,United States,8.59,1.32,7.58,7.58,2025-08-08,2025-08-19,2025-08-08,0.330,17.41,EQUITY,Oil & Gas Midstream,3432681531,USD,0.00,72,17.40,us_market,59762982912,17.5140,21.450,14.600,NaN,0.13208,None,None,61575.999488,242.000000,1.471800e+10
3,Pfizer Inc.,Pfizer,PFE,https://www.pfizer.com,NYQ,United States,4.69,1.72,7.29,7.29,2025-07-25,2025-09-02,2025-07-25,0.430,23.60,EQUITY,Drug Manufacturers - General,5685550500,USD,23.73,252,24.09,us_market,134178996224,24.5680,30.430,20.920,6.0,0.12168,None,None,62044.000256,13249.000448,2.450600e+10


In [ ]:
data

In [145]:
# 📊 Create overview DataFrame
df = pd.DataFrame(data)
df

""


In [ ]:
# 🔄 Pivot examples for analysis
pivot_sector = df.pivot_table(
    values="Annual Net (M)",
    index="Sector",
    aggfunc="sum"
).sort_values("Annual Net (M)", ascending=False)

pivot_sector


In [ ]:
# TODO: Remove obsolete
# 💰 Download dividend & yield info from Yahoo Finance
log("Downloading dividend info from Yahoo Finance...", "⬇️")
high_dividend_tickers = pruned_tickers[:50]
data = []
for ticker_symbol in with_progress(high_dividend_tickers, desc="Downloading stock information"):
    try:
        stock = get_stock(ticker_symbol)
        stock_info_json = stock.info
        dividends = stock.dividends

        dividend_yield = stock_info_json.get("dividendYield") or 0
        five_year_yield = stock_info_json.get("fiveYearAvgDividendYield") or 0
        currency = stock_info_json.get("currency") or "USD"
        annual_div = dividends[-252:].sum() if not dividends.empty else 0

        data.append({
            "Ticker": ticker_symbol,
            "Name": stock_info_json.get("longName"),
            "Sector": universe_df.loc[universe_df["Ticker"]==ticker_symbol, "Sector"].values[0],
            "Currency": currency,
            "Dividend Yield %": round(dividend_yield*100, 2),
            "5Y Avg Yield %": round(five_year_yield, 2),
            "Annual Dividend": round(annual_div, 2)
        })
        #log(f"Processed {ticker_symbol}", "✅")
    except Exception as e:
        log(f"Failed for {ticker_symbol}: {e}", "❌")


In [ ]:
# TODO: Throwaway code
vym = yf.Ticker("VYM")
vym_holdings = vym.funds_holdings
top50 = vym_holdings.head(50)

universe_df = pd.DataFrame({
    "Ticker": top50["symbol"],
    "Name": top50["holdingName"],
    "Sector": "High Dividend"
})
log(f"Collected {len(universe_df)} tickers from VYM ETF", "📊")
universe_df


In [ ]:
# TODO: Throwaway code
test_url = "https://query2.finance.yahoo.com/v1/finance/screener/predefined/saved?scrIds=day_gainers&count=5"
print(requests.get(test_url, headers={"User-Agent": "Mozilla/5.0"}).json())

In [ ]:
# TODO: Possibly throwaway code
# 📈 Add yield % if present
if "dividendYield" in df.columns:
    df["Dividend Yield %"] = (df["dividendYield"] * 100).round(2)

if "fiveYearAvgDividendYield" in df.columns:
    df["5Y Avg Yield %"] = (df["fiveYearAvgDividendYield"] * 100).round(2)

# 🧮 Per-share dividend
if "dividendRate" in df.columns:
    df["Annual Dividend (per share)"] = df["dividendRate"].round(2)

# 🏢 Company-wide totals (USD M)
if "dividendRate" in df.columns and "sharesOutstanding" in df.columns:
    df["Annual Gross (USD M)"] = (
        df["dividendRate"] * df["sharesOutstanding"] / 1_000_000
    ).round(2)
    df["Annual Net (USD M)"] = (df["Annual Gross (USD M)"] * 0.7).round(2)

log("Final dividend overview with explicit units ready", "📈")
df


# 🐞 Troubleshooting

In [71]:
def pretty_print_json(data: dict) -> str:
    """Return JSON string with indentation and sorted keys."""
    return json.dumps(data, indent=3, sort_keys=True, default=str)

from IPython.display import JSON

def pretty_print_json_new(data: dict):
    """Display JSON with indentation in Jupyter."""
    json_str = json.dumps(data, indent=2, sort_keys=True, default=str)
    display(JSON(json.loads(json_str)))  # still collapsible & colored


In [218]:
# 🐞 Troubleshoot WMT, GOOG, AG
test_tickers = ["WMT", "GOOG", "AG", "BBD", "NOK", "NDA-DK.CO", "NDA-FI.HE"]

stocks: List[Ticker] = []
for ticker_symbol in test_tickers:
    stock = get_stock(ticker_symbol)
    stocks.append(stock)
    print_stock_info(stock)
stocks_pd = stocks_to_pd(stocks)
stocks_pd


{
   "52WeekChange": 0.33610964,
   "SandP52WeekChange": 0.15519178,
   "address1": "702 South West 8th Street",
   "ask": 104.9,
   "askSize": 54,
   "auditRisk": 6,
   "averageAnalystRating": "1.4 - Strong Buy",
   "averageDailyVolume10Day": 15098420,
   "averageDailyVolume3Month": 16393050,
   "averageVolume": 16393050,
   "averageVolume10days": 15098420,
   "beta": 0.654,
   "bid": 104.55,
   "bidSize": 12,
   "boardRisk": 7,
   "bookValue": 11.299,
   "city": "Bentonville",
   "companyOfficers": [
      {
         "age": 58,
         "exercisedValue": 0,
         "fiscalYear": 2025,
         "maxAge": 1,
         "name": "Mr. C. Douglas McMillon",
         "title": "President, CEO & Director",
         "totalPay": 6249434,
         "unexercisedValue": 0,
         "yearBorn": 1966
      },
      {
         "age": 53,
         "exercisedValue": 0,
         "fiscalYear": 2025,
         "maxAge": 1,
         "name": "Mr. John David Rainey Jr.",
         "title": "Executive VP & CFO",


Converting stock list to DataFrame: 100%|██████████| 7/7 [00:00<00:00, 6998.84it/s]


,🔍,displayName,symbol,website,exchange,country,fiveYearAvgDividendYield,dividendRate,dividendYield,currentDividendYield,...,currency,ask,askSize,previousClose,market,marketCap,fiftyDayAverage,fiftyTwoWeekHigh,fiftyTwoWeekLow,boardRisk
0,Walmart Inc.,Walmart,WMT,https://corporate.walmart.com,NYQ,United States,1.37,0.94,0.90,0.90,...,USD,104.900,54,103.42,us_market,831329140736,99.0834,106.110,77.490,7.0
1,Alphabet Inc.,Alphabet,GOOG,https://abc.xyz,NMS,United States,NaN,0.84,0.34,0.34,...,USD,259.250,1,251.42,us_market,3019553636352,206.4070,253.230,142.660,NaN
2,First Majestic Silver Corp.,First Majestic Silver,AG,https://www.firstmajestic.com,NYQ,Canada,NaN,0.02,0.20,0.20,...,USD,10.190,213,10.37,us_market,4921141760,8.9316,10.900,5.090,3.0
3,Banco Bradesco S.A.,Banco Bradesco,BBD,https://banco.bradesco,NYQ,Brazil,4.49,0.17,5.19,5.14,...,USD,0.000,1792,3.21,us_market,31410081792,2.9574,3.340,1.840,NaN
4,Nokia Oyj,Nokia,NOK,https://www.nokia.com,NYQ,Finland,3.50,0.16,3.32,3.40,...,USD,0.000,970,4.61,us_market,25281744896,4.4138,5.480,3.910,NaN
5,Nordea Bank Abp,None,NDA-DK.CO,https://www.nordea.com,CPH,Finland,6.70,7.01,6.85,6.84,...,DKK,102.650,0,102.40,dk_market,354274312192,97.6408,103.800,72.040,1.0
6,Nordea Bank Abp,None,NDA-FI.HE,https://www.nordea.com,HEL,Finland,7.79,0.94,6.84,6.84,...,EUR,13.755,0,13.74,fi_market,47406219264,13.0926,13.905,9.656,1.0


In [95]:
0.17/3.28


0.05182926829268293